In [1]:
import requests
import re
from bs4 import BeautifulSoup
from urllib.request import urlopen
import os
import seaborn as sns
import charts_function_list
_,data,outputs = charts_function_list.folder_setup()

In [387]:
def main_pull(url,index_list,url_list,type_ = str):
    count = 0
    url = url
    html = urlopen(url) #get_request
    soup = BeautifulSoup(html,"lxml")
    for tr in soup.select('tbody'):
        for th in tr.find_all('th'):
            for a in th.find_all('a',href=True):
                try:
                    index_list.append(type_(a.get_text()))
                    url_list.append(base+a['href'])
   
                except:
                    pass
                    count-=1
                
                count+=1
                
                
    return count

In [493]:
base = 'https://www.pro-football-reference.com'

#soup = soup_set_up(url='https://www.pro-football-reference.com/teams/')
teams = []
urls = []
season = []
season_url = []
season_count = []
main_pull(index_list=teams,url_list=urls,url='https://www.pro-football-reference.com/teams/')

for urls_ in urls:
    pull = main_pull(index_list = season,url_list = season_url,type_=int,url=urls_)
    season_count.append(pull)
    

full_teams = [v for item,v in enumerate(teams) for x in range(season_count[item])]


                        

In [ ]:
stadium = []
stadium_url = []


for url in season_url:
    url = url
    html = urlopen(url) #get_request
    soup = BeautifulSoup(html,"lxml")
    
    texts = []

    for tr in soup.select('p'):
        for strong in tr.find_all('strong'):
            texts.append(strong.get_text())
            
    if (True in pd.Series(texts).isin(['Stadium:']).unique()) is False:
                stadium_url.append('')
                stadium.append('')
    else:


        for tr in soup.select('p'):
            for strong in tr.find_all('strong'):
                if strong.get_text()=='Stadium:':
                    for a in tr.find_all('a'):
                        stadium_url.append((a['href']))
                        stadium.append(a.get_text())
                else: 
                    pass
                
season_frame = pd.DataFrame({'Teams':full_teams,
                    'Season':season,
                    'Season URL':season_url,
                            'Stadium':stadium,
                            'Stadium_url':stadium_url})

In [710]:
os.chdir(outputs)
season_frame.to_csv('season_frame_initial.csv')

In [117]:

url = 'https://www.pro-football-reference.com/stadiums/PHO00.htm'
html = urlopen(url) #get_request
soup = BeautifulSoup(html,"lxml")

surface = []


for tr in soup.select('p'):
    
    for bold in tr.find_all('b'):
        if bold.get_text()=='Surfaces:':
            surface_text = (tr.get_text())
            print(surface_text.split(':'))
           
        #else:
         #   pass

['Surfaces', ' grass (2006-2017)']
